<a href="https://colab.research.google.com/github/0XSoumya/mistral_Rag/blob/main/basic_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mistralai
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.4 MB/s eta 0:00:00


In [3]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("")
client = Mistral(api_key=api_key)

··········


In [4]:
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

In [5]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

37

In [6]:
import time
import numpy as np

def get_text_embedding(input):
    # Perform the API call
    embeddings_batch_response = client.embeddings.create(
        model="mistral-embed",
        inputs=input
    )
    # Return the embedding from the response
    return embeddings_batch_response.data[0].embedding

# Convert text chunks to embeddings with a delay between requests
text_embeddings = []
for chunk in chunks:
    embedding = get_text_embedding(chunk)
    text_embeddings.append(embedding)
    time.sleep(3)  # Add a 1-second delay to prevent hitting rate limits

# Convert the list of embeddings to a NumPy array
text_embeddings = np.array(text_embeddings)

In [7]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [12]:
question = "who is the author?"
question_embeddings = np.array([get_text_embedding(question)])

In [13]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [14]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [15]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)

'The text is written in the first person and refers to experiences and thoughts of the author, which include working at Viaweb, being acquired by Yahoo, and starting to invest in startups. The author also mentions creating the website paulgraham.com. Therefore, the author is Paul Graham.'